## overview

In [1]:
# default package
import logging
import sys 
import os
import pathlib
import IPython
import random

In [2]:
# third party package
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import psycopg2
import sqlalchemy
from sqlalchemy.sql.expression import column,select,table
from sqlalchemy import create_engine
import sklearn.datasets

In [3]:
# my package
sys.path.append(os.path.join(pathlib.Path().resolve(),"../"))

In [4]:
# reload settings
%load_ext autoreload
%autoreload 2

In [5]:
# logger
logger=logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [6]:
## grap setting
sns.set()

In [7]:
## db setting
engine = create_engine('postgresql://postgres:postgres@db:5432/postgres')
with engine.connect() as conn:
    print(conn.closed)

False


## sql to python

In [8]:
def ct(file:str,chap_num:int,num:int,sub_num:int=None,engine:sqlalchemy.engine=engine)->None:
    """
    pubulic schema内のtableをすべて削除してから、指定のtable作成
    """
    if sub_num==None: 
        filename=f"../code-ssql2/ddl/ssql2-{file}{chap_num:02}_{num:02}"
    else:
        filename=f"../code-ssql2/ddl/ssql2-{file}{chap_num:02}_{num:02}_{sub_num}"
        
    with open(filename,"r") as f:
        stmt=f.read() 
    with engine.connect() as conn:
        conn.execute("drop schema public cascade; create schema public;")
        conn.execute(stmt)

In [9]:
def exe(stmt:str,engine:sqlalchemy.engine=engine):
    """
    sql実行
    """
    with engine.connect() as conn:
        query=conn.execute(stmt)
        try:
            result=query.fetchall()
            return result
        except:
            return None

In [10]:
def exe_df(stmt:str,engine:sqlalchemy.engine=engine)->pd.DataFrame:
    """
    sql(select)を実行してdataframeを返す
    """
    with engine.connect() as conn:
        df=pd.read_sql(stmt,conn)
    return df 

## List

In [11]:
ct("List",8,4)
exe("""
select 日付,名前,メモ
from 家計簿
join 費目
on 家計簿.費目ID=費目.ID
""")

[(datetime.date(2018, 2, 3), '食費', 'カフェラテを購入'),
 (datetime.date(2018, 2, 5), '食費', '昼食（日の出食堂）'),
 (datetime.date(2018, 2, 10), '給料', '1月の給料')]

In [12]:
ct("List",8,6)
exe("""
select 日付,家計簿.メモ,費目.メモ
from 家計簿
join 費目
on 家計簿.費目ID=費目.ID
""")

[(datetime.date(2018, 2, 3), 'カフェラテを購入', '食事代（ただし飲み会などの外食を除く）'),
 (datetime.date(2018, 2, 5), '昼食（日の出食堂）', '食事代（ただし飲み会などの外食を除く）'),
 (datetime.date(2018, 2, 10), '1月の給料', '給与や賞与が入った')]

In [13]:
ct("List",8,8)
exe("""
select 日付,費目.名前, 経費区分.名称
from 家計簿
join 費目
on 家計簿.費目ID=費目.ID
join 経費区分
on 費目.経費区分ID=経費区分.ID
""")

[(datetime.date(2018, 2, 5), '食費', '支出'),
 (datetime.date(2018, 2, 3), '食費', '支出'),
 (datetime.date(2018, 2, 10), '給料', '収入')]

In [14]:
ct("List",8,9)
exe("""
select 日付,費目.名前,費目.経費区分ID
from 家計簿
join
(select * from 費目
where 経費区分ID=1) as 費目
on 家計簿.費目ID=費目.ID
""")

[(datetime.date(2018, 2, 3), '食費', 1), (datetime.date(2018, 2, 5), '食費', 1)]

## Q

In [15]:
ct("Q",8,2,1)
exe("""
select 社員番号,社員.名前,部署.名前
from 社員
join 部署
on 社員.部署id=部署.部署id
""")

[('12459040', '宇多田定一', '開発部'), ('21000021', '菅原拓真', '開発部')]

In [16]:
ct("Q",8,2,2)
exe("""
select A.社員番号,A.名前,B.名前
from 社員 as A
left join 社員 as B
on A.上司id=B.社員番号
""")

[('21000021', '菅原拓真', '宇多田定一'), ('12459040', '宇多田定一', None)]

In [17]:
ct("Q",8,2,3)
exe("""
select 社員番号,社員.名前,部署.名前,支店.名前
from 社員
join 部署
on 社員.部署id=部署.部署id
join 支店
on 社員.勤務地id=支店.支店id
""")

[('12459040', '宇多田定一', '開発部', '東京'), ('21000021', '菅原拓真', '開発部', '東京')]

In [18]:
ct("Q",8,2,4)
exe("""
select 支店.支店id,支店.名前,A.社員数,社員.名前
from 支店 
join 
(select 勤務地id,count(*) as 社員数
from 社員
group by 勤務地id) as A
on 支店.支店id=A.勤務地id
join 社員
on 支店.支店長id=社員.社員番号
""")

[(12, '東京', 2, '宇多田定一')]

In [19]:
ct("Q",8,2,5)
exe("""
select * from 
(select A.社員番号,A.名前,
A.勤務地id as 本人勤務地,B.勤務地id as 上司勤務地
from 社員 as A
join 社員 as B
on A.上司id=B.社員番号) as C
where C.本人勤務地<>C.上司勤務地
""")

[('61381041', '大江岳人', 12, 24)]